<a href="https://colab.research.google.com/github/TranQuocViet236/Codie_in_blog_AnhKhanh/blob/master/Word2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
One-hot vector: [0,0,...,1,...,0,0]
index of the term has 1's value is index of that word in a set of words
'''

from sklearn.preprocessing import LabelEncoder

#Assign indexes for classes using LabelEncoder

le = LabelEncoder()
words = ['anh', 'em', 'gia đình', 'bạn bè', 'anh', 'em']
# le.fit(words)
# print(f'Class of words: {le.classes_}')
#
# #Convert to number:
# x = le.transform(words)
# print(f'Convert to number: {x}')

num_words = le.fit_transform(words)
print(num_words)

print(f'Invert into classes: {le.inverse_transform(num_words)}')

print(len(le.classes_))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ohe = OneHotEncoder()
classes_indices = list(zip(le.classes_, np.arange(len(le.classes_))))
print(f'Classes_indices: {classes_indices}')

ohe.fit(classes_indices)

print(f'One-hot categories and indices: {ohe.categories_}')

#Convert list words into One-hot
words_indices = list(zip(words, num_words))
print('Words and corresponding indices: ', words_indices)

one_hot = ohe.transform(words_indices).toarray()

print('Transform words into one-hot matrices: \n', one_hot)

print('Inverse transform to catefories from one hot matrix:\n', ohe.inverse_transform(one_hot))

WORD EMBEDDING

In [ ]:
!pip install  underthesea
import scipy.linalg as ln
import numpy as np
from underthesea import word_tokenize

sentence = 'Khoa học dữ liệu mà một lĩnh vực đòi hỏi kiến thức về toán và lập trình'
token = word_tokenize(sentence)
print('tokenization of sentences: ', token)

In [ ]:
from scipy.sparse import coo_matrix
#Create coherence matrix into sparse by declaring non-zero position of x and y axes
row = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
col = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
data = [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

X = coo_matrix((data, (row, col)), shape = (15,15)).toarray()
X

In [ ]:
# Perform egeneracy analysis
U, S_diag, V = ln.svd(X)
print('Shape of U', U.shape)
print('Length of diagonal: ', len(S_diag))
print('Shape of V: ', V.shape)


In [ ]:
import numpy as np
S_truncate = np.zeros((6,15))

np.fill_diagonal(S_truncate, S_diag[:6])
print('S truncate: \n', S_truncate)
print('Word Embedding 6 dimensionality: \n ', np.dot(S_truncate, V))

In [ ]:
from keras.layers import Dense, Input
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop, Adam

In [ ]:
def auto_encoder(input_unit, hidden_unit):
  model = Sequential()
  model.add(Dense(input_unit, input_shape= (15,), activation='relu'))
  model.add(Dense(hidden_unit, activation='relu'))
  model.add(Dense(input_unit, activation = 'softmax'))
  model.compile(loss='categorical_crossentropy', optimizer= Adam(), metrics=['acc'])
  model.summary()
  return model

In [ ]:
model_auto = auto_encoder(input_unit=15, hidden_unit=6)

In [ ]:
model_auto.fit(X, X, epochs=5, batch_size=3)

In [ ]:
embedding_matrix = model_auto.layers[2].get_weights()[0]
bias = model_auto.layers[2].get_weights()[1]

print('Shape of embedding_matrix: ', embedding_matrix.shape)
print('Embedding_matrix: \n', embedding_matrix)
print('Shape of bias matrix: ', bias.shape)
print('Bias matrix: \n', bias)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm

def cosine(x,y):
  cos_sim = np.dot(x,y)/(norm(x)*norm(y))
  return cos_sim
# Vector represents 'khoa học'
e0 = list(embedding_matrix[:,0])
e1 = list(embedding_matrix[:, 1])

#Quan hệ tương qua ngữ nghĩa giữa từ khoa học và dữ liệu
cosine(e0, e1)

print(cosine(e0,e1))


In [ ]:
# Từ có khoản cách lớn nhất với từ khoa học theo thứ tự
cosines = [cosine(e0, embedding_matrix[:,i]) for i in np.arange(15)]

print('cosine: ', cosines)
np.argsort([cosine(e0, embedding_matrix[:,i]) for i in np.arange(15)])[::-1]


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence
from nltk.corpus import gutenberg
from string import punctuation
import nltk

nltk.download('gutenberg')
nltk.download('punkt')

norm_bible = gutenberg.sents('/content/gdrive/MyDrive/ML/Embedding_word2vec/bible-kjv.txt')
norm_bible = [' '.join(doc) for doc in norm_bible]

In [ ]:
print(norm_bible)

In [ ]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_bible)

In [ ]:
word2id = tokenizer.word_index

In [ ]:
word2id

In [ ]:
# Build vocabulary of unique words
word2id['PAD'] = 0
id2word = {v:k for k,v in word2id.items()}
vocab_size = len(word2id)

In [ ]:
vocab_size

In [ ]:
print('Vocab sample: ', list(word2id.items())[:10])

In [ ]:
#Mã hóa toàn bộ các câu văn bằng văn bản
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in norm_bible]

In [ ]:
print(wids[:5])

In [ ]:
# Xác định context -> target

import numpy as np

def generate_context_word_pairs(corpus, window_size, vocab_size):
  context_length = window_size*2
  for words in corpus:
    sentence_length = len(words)
    for index, word in enumerate(words):
      context_words = []
      label_word = []

      #Start index of context
      start = index - window_size
      #End index of context 
      end = index + window_size +1
      #List of context_words
      context_words.append([words[i] for i in range(start, end) if 0 <=i< sentence_length and i != index])
      label_word.append(word)
      x = sequence.pad_sequences(context_words, maxlen=context_length)

      y = np_utils.to_categorical(label_word, vocab_size)

      yield(x,y)

In [ ]:
i = 0
window_size = 2 # context window size
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [ ]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

embed_size = 100

#Build CBOW architecture

cbow = Sequential()
cbow.add(Embedding(input_dim= vocab_size, output_dim= embed_size, input_length= window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1),output_shape=(embed_size)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer= 'rmsprop')
print(cbow.summary())

In [ ]:
print('Number of window: ', len(wids))

In [ ]:
for epoch in range(1,6):
  loss = 0
  i = 0
  for x,y in generate_context_word_pairs(corpus=wids[:100], window_size=window_size, vocab_size=vocab_size):
    i += 1
    loss += cbow.train_on_batch(x,y)

    if i % 500 ==0:
      print(f'Processed {i} (context,  word) pairs')

  print('Epoch:', epoch, '\tloss: ', loss)
    

In [ ]:
import pandas as pd
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

In [ ]:
#visualize model structure
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

In [ ]:
from keras.preprocessing.sequence import skipgrams

#generate skip-grams
skip_grams = [skipgrams(wids, vocabulary_size= vocab_size, window_size= window_size ) for wid in wids[:100] ]

#view sample skip-grams
pairs, labels = skip_grams[0][0], skip_grams[0][1]

for i in range(10):
  print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
    id2word[pairs[i][0]], pairs[i][0],
    id2word[pairs[i][1]], pairs[i][1],
    labels[i]))

In [ ]:
# Build neron network

from keras.layers import Input, Dot, dot, concatenate
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model, Input

#Build skip-gram architecture
word_input = Input(shape = (1,))
word_embed = Embedding(vocab_size, embed_size,
                         embeddings_initializer="glorot_uniform",
                         input_length=1, name = 'word_embedding')(word_input)
word_output = Reshape((embed_size, ))(word_embed)
word_model = Model(word_input, word_output)

In [ ]:
print('word_model: \n', word_model.summary())

In [ ]:
context_input = Input(shape = (1,))
context_embed = Embedding(vocab_size, embed_size,
                  embeddings_initializer="glorot_uniform",
                  input_length=1, name = 'context_embedding')(context_input)
context_output = Reshape((embed_size,))(context_embed)
context_model = Model(context_input, context_output)

In [ ]:
print('context_model: \n', context_model.summary())

In [ ]:
concate = dot([word_output, context_output], axes = -1)
dense = Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid")(concate)
model = Model(inputs = [word_input, context_input], outputs = dense)
model.compile(loss="mean_squared_error", optimizer="rmsprop")

In [ ]:
#view model summary 
print('model merge word and context: \n', model.summary())

In [ ]:
#Visualize model structure

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model,show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

In [ ]:




#BIỂU DIỄN T-SNE: giảm chiều dữ liệu

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

